*This code calculates the distance of each snail occurrence point to urban areas (defined by a combination of the UN, Argentina...)*

In [ ]:
# this code will create rasters that is distance to urban edges in Brazil
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

**SET UP DATA**

In [ ]:
## set up the variables to include in the function
#Brazil feature
region = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0').filter(ee.Filter.equals('ADM0_NAME', 'Brazil'));


# Read in snail points
all_sdm_points = ee.FeatureCollection('users/cglidden/all_points_schisto_sdm')

def buffer_points(radius, bounds):
    def buffer_feature(pt):
        pt = ee.Feature(pt)
        return ee.Algorithms.If(bounds, pt.buffer(radius).bounds(), pt.buffer(radius))
    return buffer_feature

# Asset of regions for which you want to calculate statistics - radius of 1km2
bufferedPoints = all_sdm_points.map(buffer_points(500, True))

# Load the image collection
worldPop = (ee.ImageCollection("WorldPop/GP/100m/pop")
            .filterMetadata('country', 'equals', 'BRA')
            .toBands()
            .unmask(ee.Image().toByte().paint(region.geometry(), 1).multiply(0))
            .clip(region))

# Convert WP to 1km
projection = worldPop.projection()
projectionAt1k = projection.atScale(1000)

pop1k = (worldPop.reduceResolution(
            reducer=ee.Reducer.sum().unweighted(),
            maxPixels=1024)
        .reproject(crs=projectionAt1k))



**DEFINE FUNCTION TO GET CONTIGUOUS URBAN PIXELS FILTERING BY TOTAL POPULATION CUTOFF (2.5k, 150k)**

In [ ]:
def mapUrban(year):

    yearString = ee.Number(year).toInt().format()
    band_name = ee.String("BRA_").cat(yearString).cat("_population")

    urbanThreshold = 300
    urbanMax = 1500

    # for medium to high pop
    urbanMask1 = pop1k.select([band_name]).gte(urbanThreshold).And(pop1k.select([band_name]).lt(urbanMax));
    urbanMask1 = urbanMask1.updateMask(urbanMask1.neq(0));

    # for high pop
    urbanMask2 = pop1k.select([band_name]).gte(urbanMax);
    urbanMask2 = urbanMask2.updateMask(urbanMask2.neq(0));

    urban1vect = urbanMask1.select([band_name]).reduceToVectors(
       geometry = region.geometry(),
        crs=pop1k.projection(),
        scale=1000,
      geometryType='polygon',
        eightConnected=True,
        labelProperty='zone',
        bestEffort=False,
        maxPixels=1e13,
        tileScale=16
    )

    urban2vect = urbanMask2.select([band_name]).reduceToVectors(
        geometry= region.geometry(),
        crs=pop1k.projection(),
        scale=1000,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='zone',
        bestEffort=False,
        maxPixels=1e13,
        tileScale=16
    )

    # now get population per feature
    reducer = ee.Reducer.sum();
    pop_sum_features1 = (pop1k.select([band_name]).reduceRegions(
                      collection = urban1vect,
                      reducer = reducer,
                      scale = 1000,
                      crs = 'EPSG:4326',
                      tileScale = 16));


    pop_sum_features2 = (pop1k.select([band_name]).reduceRegions(
                      collection = urban2vect,
                      reducer = reducer,
                      scale = 1000,
                      crs = 'EPSG:4326',
                      tileScale = 16));

    # filter features
    pop_sum_features_final1 = (pop_sum_features1
                              .filter(ee.Filter.gte('sum', 2500)));

    pop_sum_features_final2 = (pop_sum_features2
                              .filter(ee.Filter.gte('sum', 150000)));

    # convert back to binary image
    final_int_urban = (pop_sum_features_final1
                       .filter(ee.Filter.notNull(['sum']))
                       .reduceToImage(
                          properties = ['sum'],
                          reducer = ee.Reducer.first())
                       .rename(ee.String("contig_Iurban_").cat(ee.Number(year).toInt().format()))
                       .unmask(0).clip(region).gt(0));

    final_high_urban = (pop_sum_features_final2
                       .filter(ee.Filter.notNull(['sum']))
                       .reduceToImage(
                          properties = ['sum'],
                          reducer = ee.Reducer.first())
                       .rename(ee.String("contig_Hurban_").cat(ee.Number(year).toInt().format()))
                       .unmask(0).clip(region).gt(0));

    return ee.Image(final_int_urban.addBands(final_high_urban))

# Define a function to remove first two characters from band name
def remove_prefix(band_name):
    return ee.String(band_name).slice(2)




**Export urban boundaries in chunks bc memory intensive**








* 2000 - 2002

In [ ]:
####### break up export bc of memory issues
# Define the years as a list of strings - 2000 - 2006
years_2002 = ee.List.sequence(2000, 2002) # Modify this list as needed

region_images2002 = ee.ImageCollection(years_2002.map(mapUrban))
region_image2002 = region_images2002.toBands()

# Get the band names
band_names2002 = region_image2002.bandNames()

# Apply the function to each band name
new_band_names2002 = band_names2002.map(remove_prefix)

# Rename the bands
renamed_image2002 = region_image2002.select(band_names2002, new_band_names2002)
# print(region_images)

# Define the asset ID where you want to save the collection
asset_id2002 = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2002'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_image2002,
    description ='contigUrban2002',
    assetId=asset_id2002,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

*   2003 - 2006

In [ ]:
# Define the years as a list of strings - 2000 - 2006
years_2006 = ee.List.sequence(2003, 2006) # Modify this list as needed

region_images2006 = ee.ImageCollection(years_2006.map(mapUrban))
region_image2006 = region_images2006.toBands()

# Get the band names
band_names2006 = region_image2006.bandNames()

# Apply the function to each band name
new_band_names2006 = band_names2006.map(remove_prefix)

# Rename the bands
renamed_image2006 = region_image2006.select(band_names2006, new_band_names2006)
# print(region_images)

# Define the asset ID where you want to save the collection
asset_id2006 = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2006'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_image2006,
    description ='contigUrban2006',
    assetId=asset_id2006,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

*   2007 - 2008

In [ ]:
####### break up export bc of memory issues - 2007:2008
years_2008 = ee.List.sequence(2007, 2008) # Modify this list as needed

region_images2008 = ee.ImageCollection(years_2008.map(mapUrban))
region_image2008 = region_images2008.toBands()

# updated bands
band_names2008 = region_image2008.bandNames()
new_band_names2008 = band_names2008.map(remove_prefix)
renamed_image2008 = region_image2008.select(band_names2008, new_band_names2008)

# asset ID
asset_id2008 = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2008'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_image2008,
    description ='contigUrban2008',
    assetId = asset_id2008,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

*   2009 - 2013

In [ ]:
####### break up export bc of memory issues - 2009:2010
years_2013 = ee.List.sequence(2009, 2013) # Modify this list as needed

region_images2013 = ee.ImageCollection(years_2013.map(mapUrban))
region_image2013 = region_images2013.toBands()

# updated bands
band_names2013 = region_image2013.bandNames()
new_band_names2013 = band_names2013.map(remove_prefix)
renamed_image2013 = region_image2013.select(band_names2013, new_band_names2013)

# asset ID
asset_id2013 = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2013'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_image2013,
    description ='contigUrban2013',
    assetId = asset_id2013,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

*   2014 - 2017

In [ ]:
####### break up export bc of memory issues - 2014:2017
years_2017 = ee.List.sequence(2014, 2017) # Modify this list as needed

region_images2017 = ee.ImageCollection(years_2017.map(mapUrban))
region_image2017 = region_images2017.toBands()

# updated bands
band_names2017 = region_image2017.bandNames()
new_band_names2017 = band_names2017.map(remove_prefix)
renamed_image2017 = region_image2017.select(band_names2017, new_band_names2017)

# asset ID
asset_id2017 = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2017'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_image2017,
    description ='contigUrban2017',
    assetId = asset_id2017,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()



*   2018-2020



In [ ]:
####### break up export bc of memory issues - 2018:2020
years_2020 = ee.List.sequence(2018, 2020) # Modify this list as needed

region_images2020 = ee.ImageCollection(years_2020.map(mapUrban))
region_image2020 = region_images2020.toBands()

# updated bands
band_names2020 = region_image2020.bandNames()
new_band_names2020 = band_names2020.map(remove_prefix)
renamed_image2020 = region_image2020.select(band_names2020, new_band_names2020)

# asset ID
asset_id2020 = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2020'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_image2020,
    description ='contigUrban2020',
    assetId = asset_id2020,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

**FINAL CUMULATIVE COST MAPPING GeoTiffs** -- need to consolidate in image collection

In [ ]:
# read in images from mapUrban
contigUrban2002 = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2002").clip(region))
contigUrban2006 = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2006").clip(region))
contigUrban2008 = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2008").clip(region))
contigUrban2013 = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2013").clip(region))
contigUrban2017 = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2017").clip(region))
contigUrban2020 = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/contigUrbanAR/contigUrban2020").clip(region))

contigUrban = (contigUrban2002.addBands(contigUrban2006).addBands(contigUrban2008).addBands(contigUrban2013).addBands(contigUrban2017)
              .addBands(contigUrban2020));


In [ ]:
##### FINAL COST MAPPING
def mapCost (year):

    yearString = ee.Number(year).toInt().format();
    band_name_int = ee.String("contig_Iurban_").cat(yearString);
    band_name_high = ee.String("contig_Hurban_").cat(yearString);

    final_int_urban_binary = contigUrban.select([band_name_int])
    final_high_urban_binary = contigUrban.select([band_name_high])

    # set masked values to 50km
    overUrban = ee.Image().toByte().paint(region.geometry(), 50000)

    # create even surface
    landscape = ee.Image().toByte().paint(region.geometry(), 1)

    # int urban cross mapping
    final_int_urban_binary_v2 = final_int_urban_binary.gt(0) # maybe unneccesary
    cumulativeCostInt0 = (landscape.cumulativeCost(
                            source=final_int_urban_binary_v2,
                            maxDistance= 50 * 1000,
                            geodeticDistance = False)
                         .rename(ee.String("inte_").cat(ee.Number(year).toInt().format()))
                         .unmask(overUrban))

    # convert anything greater than 50km to 50km
    condition1 = cumulativeCostInt0.lte(50000); #50,000 meters = 50 km
    cumulativeCostInt = cumulativeCostInt0.updateMask(condition1).unmask(50000).clip(region);

    # repeat for high density
    final_high_urban_binary_v2 = final_high_urban_binary.gt(0)
    cumulativeCostHigh0 = (landscape.cumulativeCost(
                            source=final_high_urban_binary_v2,
                            maxDistance= 50 * 1000,
                            geodeticDistance = False)
                         .rename(ee.String("high_").cat(ee.Number(year).toInt().format()))
                         .unmask(overUrban))

    # convert anything greater than 50km to 50km
    condition2 = cumulativeCostHigh0.lte(50000);
    cumulativeCostHigh = cumulativeCostHigh0.updateMask(condition2).unmask(50000).clip(region);

    return ee.Image(cumulativeCostInt.addBands(cumulativeCostHigh))

# Define a function to remove index from band name
def remove_index(band_name):
    return ee.String(band_name).slice(-9)

**TRY TO RUN IN ONE OUTPUT**

In [ ]:
years = ee.List.sequence(2000, 2020) # Modify this list as needed

cc_images = ee.ImageCollection(years.map(mapCost))
cc_image = cc_images.toBands()

# updated bands
cc_names = cc_image.bandNames()
new_cc_names = cc_names.map(remove_index)
renamed_cc = cc_image.select(cc_names, new_cc_names)

# asset ID
cc_id = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/urbanCostMapAR'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_cc,
    description ='urbanCostMapAR',
    assetId = cc_id,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

**Read in cost mapping data**

In [ ]:
# read in images from mapUrban
cost_maps = (ee.Image("projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/urbanCostMapAR").clip(region))

**Map over feature collection**

In [ ]:
#### export feature collection 1km around snails
# worth it to map it so we are only getting data on the ones per year?

reducer2 = ee.Reducer.mean()

finalFeature = cost_maps.reduceRegions(
      collection = bufferedPoints,
                      reducer = reducer2,
                      scale = 1000,
                      crs = 'EPSG:4326',
                      tileScale = 16);

# Export the image sample feature collection to Drive as a CSV file.
task = ee.batch.Export.table.toDrive(
    collection=finalFeature,
    description='schisto_urbanCC_AR_oct102023',
    folder='final_schisto_data',
    fileFormat='CSV',
)
task.start()

**NOW READ IN CSV & CLEAN**

In [ ]:
# acess the file & set up file
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# read in data
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/GEEexports brazil_schisto_snails/final_schisto_data/schisto_urbanCC_AR_oct102023.csv')
# print(df.head())
print(df.columns)

In [ ]:
# subset data to make it easier to switch from wide to long
desired_indices_high = list(range(2, 23)) + [45] + [48]
df_high = df.iloc[:, desired_indices_high] # might have to update this
print(df_high.columns)

# swtich high urban from wide to long
desired_indices_int = list(range(23, 44)) + [45] + [48]
df_int = df.iloc[:, desired_indices_int] # might have to update this
print(df_int.columns)

In [ ]:
#### now convert wide to long dataset, merge data, and export (print len to make sure merged okay)

df_high_long = pd.melt(df_high, id_vars=['row_code', 'year'], var_name='year2', value_name='dist_high_urbanAR')
df_high_long = df_high_long.dropna()
df_high_long['year2'] = df_high_long['year2'].str.extract(r'(\d+)')
df_high_long['year2'] = pd.to_numeric(df_high_long['year2'])
df_high_long = df_high_long[(df_high_long['year'] == df_high_long['year2'])]
df_high_long = df_high_long.drop(columns=['year2'])
print(df_high_long.head())
print(len(df_high_long)) # no of rows

df_int_long = pd.melt(df_int, id_vars=['row_code', 'year'], var_name='year2', value_name='dist_int_urbanAR')
df_int_long = df_int_long.dropna()
df_int_long['year2'] = df_int_long['year2'].str.extract(r'(\d+)')
df_int_long['year2'] = pd.to_numeric(df_int_long['year2'])
df_int_long = df_int_long[(df_int_long['year'] == df_int_long['year2'])]
df_int_long = df_int_long.drop(columns=['year2'])
print(df_int_long.head())
print(len(df_int_long)) # no of rows


In [ ]:
## save code
merged_df = pd.merge(df_high_long, df_int_long, on=['row_code','year'], how='inner')
print(len(merged_df))
print(merged_df.head)

merged_df.to_csv('/content/drive/MyDrive/GEEexports brazil_schisto_snails/final_schisto_data/cleaned_schisto_urbanCC_AR_oct102023.csv', index=False)
